In [2]:
!pip install html2text

In [3]:
import re
from nltk import ngrams
from collections import Counter
import html2text
import random
import pickle

In [4]:
class ngram:
    def __init__(self, filename,n): 
        self.__data = []
        self.__n = n
        self.model = {}
        self.__string_data = self.__readfile(filename)
        print("Syllable in progress...")
        self.__data = self.syllables(self.__string_data)
        self.__create_ngram_probability()
        self.__gt = 0

    def __readfile(self,filename):
        
        print("Reading {} file...".format(filename))
        
        f = open(filename,"r",encoding="utf8",errors='ignore')
        #f = open(filename,"r")
        data = f.read()
        data = html2text.html2text(data)
        data = data.lower()
        data = re.sub(r'[^a-zçğıöşü ]', '', data)
        
        f.close()
        return data

    def __create_ngram_probability(self):

        print("n-gram model creating...")

        self.__ngram = ngrams(self.__data, self.__n)

        self.__counter = Counter(self.__ngram)
        self.GT_smoothing()
        
        t_gram = Counter(ngrams(self.__data, self.__n-1))

        if self.__n == 1:
            for key in self.__counter.keys():
                self.model[key] = self.__counter.get(key) / len(self.__counter.keys())
        else:   
            for key in self.__counter.keys():
                self.model[key] = self.__counter.get(key) / t_gram.get(key[0:-1])

        self.model['UNK'] = 0
    
    def GT_smoothing(self):

        count = 0
        gt_dict = {}
        for key,value in self.__counter.items():
            if value == 1:
                count +=1
            if gt_dict.get(value) == None:
                gt_dict[value] = 1
            else:
                gt_dict[value] = gt_dict.get(value) +1
        
        for key,value in self.__counter.items():
            if gt_dict.get(value + 1) != None : 
                self.__counter[key] = (value +1) * (gt_dict.get(value +1) / gt_dict.get(value) )
            
        self.__gt = count / len(self.__counter.keys())

    def test_perplexity(self,filename):

        test_data = self.__readfile(filename)       
        res_dict = {}
        test_data = test_data.split(" ")
        
        print("Perpleixty calculating...")

        for test in test_data:
            result = 1.0
            sly = [" "]
            sly.extend(self.syllables(test))
            sly.extend([" "])

            for i in range(len(sly) - self.__n +1):
                temp = sly[i:self.__n+i]
                if(self.model.get(tuple(temp)) == None):
                    flag = False
                    for t in temp:
                        if t in self.__data:
                            flag = True
                            break
                    # if we see the same syllabus before it means that this
                    # combination doesnt happened before so we use GT smoothing variable
                    if flag:
                        result *= self.__gt / len(self.model)
                    else:
                        self.model['UNK'] = self.model.get('UNK') +1
                        result *= (self.model.get('UNK')) / len(self.model)
                else:
                    result *= self.model.get(tuple(temp))
            if self.__n > len(sly):
                res = 0
                t = 0
                for key in self.__counter.keys():
                    if str(tuple(sly))[1:-1] in str(key) :
                        res += self.__counter.get(key)
                    if str(tuple(sly[:-1]))[1:-1] in str(key):
                        t += self.__counter.get(key)
                if (res == 0) or (t == 0):
                    flag = False
                    for t in temp:
                        if t in self.__data:
                            flag = True
                            break
                    if flag:
                        result *= self.__gt / len(self.model)
                    else:
                        self.model['UNK'] = self.model.get('UNK') +1
                        result = (self.model.get('UNK')) / len(self.model) 
                else:
                    result = res/t
            res_dict[''.join(sly[1:-1])] = (1 / result)**(1/ len(sly))

        return res_dict 

    # choose one of the 5 syllabus and produce a sentence
    def generate_sentence(self,res = (' ',)):

        print("Generate sentence:")
        result = ""
        res = self.__generate_sentence(res)
        for t in res[1:]:
            result += t
        while(True):
            res = self.__generate_sentence(res[1:])
            for t in res[self.__n-1:]:
                 result += t 
            if ' ' in result:
                break
        print(result.split(" ")[0])

    def __generate_sentence(self,syllable):

        keys = self.__counter.keys()
        count = [0]*5
        res = [(" "),(" "),(" "),(" "),(" ")]
        
        for key in keys:
            if (str(key).startswith(str(syllable)[:-2])):

                for i in range(0,5):
                    if self.__counter.get(key) > count[i]:
                        res[i:5-1] = res[i+1:5]
                        count[i:5-1] = count[i+1:5]
                        count[i] = self.__counter.get(key)
                        res[i] = key
                        break
        return random.choice(res)

    def syllables(self,word):
        
        syllables = []
        #print("Syllable in progress...")
        
        bits = ''.join(['1' if l in 'aeıioöuü' else '0' for l in word])

        seperators = (
            ('101', 1),
            ('1001', 2),
            ('10001', 3)
        )

        index, cut_start_pos = 0, 0

        while index < len(bits):

            for seperator_pattern, seperator_cut_pos in seperators:
                if bits[index:].startswith(seperator_pattern):

                    if (' ' in word[cut_start_pos:index + seperator_cut_pos]):

                        if word[cut_start_pos:index + seperator_cut_pos][0] == ' ':
                            syllables.append(' ')
                            syllables.append(word[cut_start_pos +1 :index + seperator_cut_pos])
                        else:
                            syllables.append(word[cut_start_pos:index + seperator_cut_pos - 1])
                            syllables.append(' ')
                    else:
                        syllables.append(word[cut_start_pos:index + seperator_cut_pos])

                    index += seperator_cut_pos
                    cut_start_pos = index
                    break

            index += 1

        syllables.append(word[cut_start_pos:])
        #print("syllables is completed")
        return syllables

In [5]:
res1 = ngram("text",1)

Reading text file...
Syllable in progress...
n-gram model creating...


In [6]:
res2 = ngram("text",2)

Reading text file...
Syllable in progress...
n-gram model creating...


In [7]:
res3 = ngram("text",3)

Reading text file...
Syllable in progress...
n-gram model creating...


In [8]:
res4 = ngram("text",4)

Reading text file...
Syllable in progress...
n-gram model creating...


In [9]:
res5 = ngram("text",5)

Reading text file...
Syllable in progress...
n-gram model creating...


In [39]:
res1.generate_sentence()

Generate sentence:
lelelela


In [41]:
res2.generate_sentence()

Generate sentence:
vekitaplaşmarafındandırtürkedenizma


In [45]:
res3.generate_sentence()

Generate sentence:
içinde


In [48]:
res4.generate_sentence()

Generate sentence:
ile


In [28]:
res5.generate_sentence()

Generate sentence:
tarafından


In [36]:
res1.test_perplexity("perplexity")

Reading perplexity file...
Perpleixty calculating...


{'orta': 0.5005046745323962,
 'italya': 1.0222430438364583,
 'coğrafi': 1.6878991909919954,
 'bölgesi': 0.9869073111108995,
 '': 0.6181324076067047,
 'veya': 0.2954989169804265,
 'sadece': 0.5690797687872495,
 'resmiidari': 3.1612918562068586,
 'birniteleme': 0.7283067283470206,
 'olmayıp': 1.607047527305015,
 'geniş': 0.6750256335452967,
 'kapsamlı': 3.5319739805427437,
 'kültürel': 3.2665157075384808,
 'tarihsel': 0.9910816238594876,
 've': 0.1797936717140122,
 'birtanımlamadır': 1.1784270301383748,
 'italyanın': 1.3154775337621145,
 'olarak': 0.4785522706409385,
 'kuzey': 1.2390875282002096,
 'kısmında': 1.5895926687962612,
 'bulunan': 0.9437968121078821,
 'coğrafialanları': 3.00963777377914,
 'ihtiva': 1.4807157882000848,
 'eder': 0.8721018129103484,
 'italyan': 1.624972124780483,
 'istatistiksel': 3.5025996605930447,
 'enstitüsü': 3.0938251826552157,
 'istat': 2.2163395847948353,
 'tarafındankullanılan': 1.4572977579748267,
 'beş': 0.9207887785526377,
 'resmi': 1.0528228561751243,